In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("reliance_data.csv", parse_dates=True)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-01-01 00:00:00+05:30,679.081997,683.152914,673.490123,675.324219,14004468,0.0,0.0
1,2020-01-02 00:00:00+05:30,676.397899,689.348791,676.397899,686.821228,17710316,0.0,0.0
2,2020-01-03 00:00:00+05:30,685.792252,689.661895,681.318729,687.648804,20984698,0.0,0.0
3,2020-01-06 00:00:00+05:30,679.976719,683.510767,670.134933,671.700684,24519177,0.0,0.0
4,2020-01-07 00:00:00+05:30,679.529381,686.463396,677.068950,682.034607,16683622,0.0,0.0


In [15]:
class Candle:
    def __init__(self, row_data: pd.Series):
        self.date = row_data.Date
        self.o = row_data.Open
        self.c = row_data.Close
        self.h = row_data.High
        self.l = row_data.Low
        self.v = row_data.Volume
        self.body = abs(self.c - self.o)
        self.range = abs(self.h - self.l)
        self.body_weight = self.body / self.range
        self.is_bullish = self.o < self.c
        self.is_bearish = self.o > self.c
        
    def __repr__(self):
        return f"{self.date}, O - {self.o}, C - {self.c}, H - {self.h}, L - {self.l}"
    
    def is_long(self, multiplier=2.0, lookback=10) -> bool:
        avg_body = get_avg_body(data, self, lookback)
        return self.body > multiplier * avg_body
    
    def is_bozo(self, proportion: float, multiplier, lookback) -> bool:
        # try:
        #     if 0 < proportion < 1:
        #         pass
        # except:
        #     raise ValueError("Proportion must be between 0 and 1")
        return self.is_long(multiplier, lookback) and self.body_weight > proportion

def get_last_n_candles(data: pd.DataFrame, candle: Candle, range: int):
    index = index_from_date(data, candle.date)  # Fixed: pass data first, then date
    start = max(0, index - range)
    return data.loc[start:index]
    
def get_avg_body(data: pd.DataFrame, candle: Candle, lookback: int) -> float:
    previous_candles = get_last_n_candles(data, candle, lookback)
    return abs(previous_candles.Close - previous_candles.Open).mean()

def index_from_date(data: pd.DataFrame, date) -> int:  # Fixed: swapped parameters and return int
    return data[data.Date == date].index[0]

print("Candle Class and helper functions initialised!")

Candle Class and helper functions initialised!


In [16]:
candle = Candle(data.loc[0])
candle.is_long()


np.False_